<div style="text-align: right"> 김영진 </div>

# Chapter 4 Numerical Computation

p1 :

- Machine Learning은 일반적으로 많은 양의 numerical computation을 요구함
  - **iterative process**를 통해 추정치를 update하는 방식으로 수학적 문제를 해결하는 알고리즘들...
  - optimization : 특정 함수를 최대 혹은 최소로 하는 변수 값을 찾는 과정
  - solving systems of linear equations
- 함수 내에 유한한 양의 메모리를 통해 정확하게 표현할 수 없는 **real number**가 포함될 경우, 수학적인 함수를 digital computer에서 실행(evaluating)하기가 어려워짐
  

## 4.1 Overflow and Underflow

p1:
- continuous math를 digital computer에서 실행시키는 가장 근본적인 어려움:
    - 무한한 실수를 유한한 개수의 bit pattern으로 표현해야 할 필요가 있음 (디지털화)
    - 결과적으로, approximation error가 발생함 : 대부분 **rounding error**
        - rounding error는 여러 operations과 혼합되어 동작할 경우, error가 축적 혹은 전파되어 전체 프로그램을 망가트리는 문제를 발생시킬 수 있음

<br />
p2:

- **underflow**
    - 숫자가 0에 가깝거나, 반올림할 때 0이 되는 경우 발생하는 문제
    - 대부분의 알고리즘에서는 **0**과, **0에 가까운 양수 혹은 음수**가 완전하게 구분되어야 할 필요성이 있음
        - *division by zero*
        - *logarithm of zero*

In [3]:
print(4/0)

ZeroDivisionError: division by zero

In [4]:
import numpy as np
print(np.log(0))

-inf


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


p3(**page 81**):

- **overflow**
    - 숫자가 양 혹은 음의 무한대에 가까울 경우 발생하는 문제
    - *non-a-number* : nan
   
<br />
p4:

$$
    \text{softmax}(x)_i = \frac{\text{exp}(x_i)} {\sum_{j=1}^n \text{exp}(x_j)}
$$

- underflow나 overflow로부터 반드시 *stabilize*해야 하는 함수 : **softmax**
    - 모든 $ x_i $ 가 동일한 상수 $ c $ 라고 가정해보자!
        - 수학적으로 softmax의 결과는 $ \frac {1} {n} $ 이어야 함
        - c가 매우 큰 음수인 경우, $ \text{exp}(c) $ 는 underflow
            - softmax의 분모가 0이 됨에 따라, softmax결과가 undefined 됨
        - c가 매우 큰 양수인 경우, $ \text{exp}(c) $ 는 overflow
            - softmax결과가 무한대가 됨에 따라, undefined 됨
    - 해결 방법 : $ x $ 를 $ z $로 대체 ($ z = x - \max_i x_i $)
        - *부호와 관계 없이 큰 수를 작은 수로 만드는 효과??*

<br />
p5: 

- *log softmax(x)* 의 분자 부분에서 발생하는 underflow
    - 상기와 동일한 방법으로 해결 가능
    
<br />
p6:

- 본 교재에서는 명시적으로 이러한 numerical consideration에 대해 자세하게 언급하지 않음
- low-level libraries 개발자는 반드시 이러한 numerical issue들을 상기하고 있어야 함
- Theano는 자동으로 numerical issue들을 감지하고, 대부분의 common numerically unstable expression 들을 stabilize함

## 4.2 Poor Conditioning

p1(**page 82**):
- *Conditioning* : inputs의 작은 변화에 대해 얼마만큼 함수의 출력이 크게 변동하는지를 나타냄
    + 작은 입력의 변동에서 함수의 값이 크게 변동될 경우, 입력의 rounding error는 큰 문제를 발생시킴

<br />
p2:

$$
    f(x) = A^{-1}x
$$

- $ A \in \mathbb{R}^{n \times n} $ : eigenvalue decomposition
- *condition number* : 가장 큰 eigenvalue와 가장 작은 eigenvalue의 비율
    + condition number가 클 경우, matrix inversion은 input의 error에 민감함(sensitivity)
    
$$
    \underset{i, j}{\operatorname{max}} \left | \frac{\lambda_i} {\lambda_j} \right |
$$

<br />
p3:

- 이러한 sensitivity는 역행렬 계산 과정에서의 rounding error로 인한 결과가 아닌 matrix 자체의 본질적인 특성임
- *Poorly conditioned* matrices는 실제 역행렬과 곱하는 과정에서 pre-existing error를 증폭하는 결과를 초래함

## 4.3 Gradient-Based Optimization

p1:
- *Optimization* : x를 수정해서 $ f(x) $를 최소화(minimizing)하거나 최대화(maximizing)하는 일련의 과정
- 대부분 Optimization은 $ f(x) $를 최소화하는 과정으로 간주함
    + 최대화는 $ -f(x) $를 최소화하는것으로 치환 가능함

<br />
p2:

- 최소화하거나 최대화하고자 하는 함수를 목적 함수(*obective function*) 혹은 *criterion*이라 함
    - *cost function*, *loss function*, *error function*이라고도 불림
    - 실제 상기의 용어들은 아주 약간의 차이를 가지고 있으나, 혼용되어 사용됨

<br />
p3:

- 함수를 최대화 혹은 최소화하는 파라미터를 $ x^* = \text{argmin}f(x) $ 로 표현함

<br />

p4(**page 83**):

- optimization과 *calculus*(미분, 적분) 개념이 어떻게 연관되어 있는가를 소개할 것임

<br />
p5:

- 함수 $ y = f(x) $가 있고, $ x $와 $ y $모두 실수라고 가정해보자!
    - 미분 : $ f'(x) $, $ \frac {dy} {dx} $
    - 함수 $ f(x) $의 미분은 점 $ x $에서의 기울기를 의미함
    - 입력에 대한 약간의 변화가 출력에 얼마만큼의 영향을 미치는지를 의미함 : $ f(x + \epsilon) \approx f(x) + \epsilon f'(x) $

<br />
p6:

- 미분(*derivative*)은 함수를 최소화하는데 유용하게 사용할 수 있음
    - y를 약간 향상시키는데(최소화하는데) x를 어떻게 움직여야 하는지 알수 있음
    - $ f(x - \epsilon \text{sign}(f'(x)))  < f(x) \text{ for small enough } \epsilon $
        - $ x $의 값을 미분 값 부호의 반대 방향으로 이동시킴으로써 $ f(x) $ 값을 줄일 수 있음!
    - *gradient descent*
    
![Figure 4.1](https://github.com/SCPSAI/Deep-Learning-Book/blob/master/images/fig_4_1.PNG?raw=true)

<br />
p7:

- $ f'(x) = 0 $ : *critical points*, *stationary points*
- *local minimum* : 모든 주변의 점보다 $ f(x) $의 값이 작은 점 $ x $
    - infinitesimal step들로 $ f(x) $를 더이상 줄일 수 없는 상태
- *local maximum* : 모둔 주변의 점보다 $ f(x) $의 값이 큰 점 $ x $
    - infinitesimal step들로 $ f(x) $를 더이상 키울 수 없는 상태
- *saddle points* : 최소값이나 최대값도 아닌 critical points

![Figure 4.2](https://github.com/SCPSAI/Deep-Learning-Book/blob/master/images/fig_4_2.PNG?raw=true)

<br />

p8(**page 84**):

- *global minimum* : $ f(x) $에 대해 가장 작은 값
    - 하나의 global minimum이 있을수도 있고, 다수의 global minimum이 있을수도 있음
    - global minimum이 아닌 local minimum이 있을 수 있음
- deep learning 관점에서 global minimum을 찾는 것이 목적
    - global minimum이 아닌 local minimum들과 saddle points는 optimization을 어렵게 하는 요소임
    - 특히, 입력이 multidimensional 일 때, 상기의 상황에 잘 직면함

![Figure 4.3](https://github.com/SCPSAI/Deep-Learning-Book/blob/master/images/fig_4_3.PNG?raw=true)

<br />
p9:

- 다수의 입력을 갖는 함수 $ f : \mathbb{R}^n \to \mathbb{R} $ 를 최소화하고자 할 경우
    - 여전히, 1개의 출력을 갖으므로 최소화 가능

<br />
p10:

- 함수가 다수의 입력을 갖을 때, 편미분(*partial derivatives*)을 적용함
    - *partial derivative* ($ \frac {\partial} {\partial x_i} f(x) $) : 
        + 점 $ x $에서 $ x_i $ 가 증가할 때 함수 $ f $가 어떻게 변하는가를 의미
    - $ f $의 gradient($ \triangledown_x f(x) $) : x의 모든 요소에 대한 partial derivative를 갖는 벡터로 구성됨
    - *critical points* : gradient vector의 모든 요소들이 0인 점

<br />

p11(**page 85**):

- *directional derivative* : 방향 $ u $ (unit vector) 에서의 함수 $ f $의 기울기 : $ u^{\top}\triangledown_xf(x) $

<br />
p12:

- 함수 $ f $를 최소화하기 위해서, 함수 $ f $를 최소화하는 방향을 찾을 필요가 있음 $ \to $ directional derivative의 사용

$$
    \underset{u, u^{\top}u=1}{\operatorname{min}} u^{\top}\triangledown_xf(x)
$$

$$
    = \underset{u, u^{\top}u=1}{\operatorname{min}} ||u||_2 ||\triangledown_xf(x)||_2 \cos \theta
$$

- $ \theta $ : $ u $ 와 gradient 사이의 각도(angle)
- $ ||u||_2 = 1 $ 로 치환하고, $ u $에 의존적이지 않은 factor들을 무시하면 $ \to \min_u \cos \theta $
    - 방향 $ u $와 gradient가 반대 방향일 때 최소화됨
    - 즉, gradient는 오르막을 가리킴 negative gradient는 내리막을 가리킴
        - 함수 $ f $를 gradient gradient의 방향으로 이동시키면, 최소화할 수 있음
        - *steepest descent*, *gradient descent*

<br />
p13:

- *steepest descent* :

$$
    x' = x - \epsilon \triangledown_xf(x)
$$

- $ \epsilon $ : learning rate
    - step size parameter (positive scalar)
    - 다양한 설정 방법이 존재함
    - 일반적으로 작은 상수(small constant)로 설정함
    - *line search* : $ f(x-\epsilon \triangledown_xf(x)) $의 값을 여러 $ \epsilon $에 대해 실험해서, 가장 작은 objective function value를 갖는 결과를 선택하는 방법

<br />

p14(**page 86**):
- gradient 벡터의 모든 요소가 0이거나 0에 가까워질 때, Steepest descent 는 수렴(converge)함
- $ \triangledown_xf(x) = 0 \text{ for } x $를 풀어서 직접적으로 critical point를 찾을 수도 있음

<br />
p15:

- gradient descent는 연속 공간(continuous space)에서의 최적화로 제한됨
    - better configurations로의 작은 움직임과 같은 gradient descent의 일반적인 개념은 이산 공간(discrete space)로 일반화시킬 수 있음
        - discrete parameter들의 objective function을 상승시키는 것 $ \to $ *hill climbing*

### 4.3.1 Beyond the Gradient: Jacobian and Hessian Matrices

p1: 

- $ f : \mathbb{R}^m \to \mathbb{R}^n $ : 다수의 입력과 출력을 갖는 함수를 최적화하고자 할 경우
    - 모든 입력과 출력 쌍에 대한 partial derivative들을 구해야 함
    - *jacobian matrix* ($ J \in \mathbb{R}^{n \times m} $) : patial derivative들로 구성된 matrix
        - $ J_{i,j} = \frac {\partial} {\partial x_j} f(x)_i $
        
<br />
p2:

- *second derivative* : 미분의 미분
    - 함수 $ f : \mathbb{R}^m \to \mathbb{R}^n $에서의 sencond derivative : $ \frac {\partial^2} {\partial x_i \partial x_j} f $
    - single dimension 함수에서의 sencond derivative : $ \frac {d^2} {dx^2} $, $ f''(x) $
    - 함수의 미분이 입력의 작은 변화에 대해 어떻게 바뀌는가를 의미함
    - 즉, gradient step이 얼마만큼 함수를 최소값으로 움직일 수 있는지를 알 수 있음
    - *curvature*를 측정하는 것으로 간주할 수 있음
    
![Figure 4.4](https://github.com/SCPSAI/Deep-Learning-Book/blob/master/images/fig_4_4.PNG?raw=true)

<br />

p3(**page87**):

- 다수의 입력을 갖는 함수가 있을 때, 다수의 second derivative가 존재함
    - *Hessian matrix*($ H(f)(x) $) : 모든 second derivative를 모아놓은 matrix
    - Jacobian의 gradient를 의미함
    
$$
    H(f)(x)_{i,j} = \frac {\partial^2} {\partial x_i \partial x_j}f(x)
$$

<br />
p4:

- 미분 연산은 교환법칙(commutative)이 성립되므로, 순서를 바꿀 수 있음

$$
    \frac {\partial^2} {\partial x_i \partial x_j}f(x) = \frac {\partial^2} {\partial x_j \partial x_i}f(x)
$$

- 따라서, Hessian matrix는 대칭(symmetric)성을 갖음
    - Hessian matrix는 real eigenvalues와 서로 직교인 eigenvectors로 분해될 수 있음
    - 특정 방향 $ d $(unit vector)로의 second derivative는 $ d^{\top}Hd $로 표현됨

    > When $ d $ is an eigenvector of $ H $, the second derivative in that direction is given by the corresponding eigenvalue. For other directions of $ d $, the directional second derivative is a weighted average of all of the eigenvalues, with weights between 0 and 1, and eigenvectors that have smaller angle with $ d $ receiving more weight.

    - maximum eigenvalue는 maximum second derivative를 결정
    - minimum eigenvalue는 minimum second derivative를 결정
    
<br />

p5(**page 88**):

- (directional) second derivative는 gradient descent step이 잘 수행되었는가를 나타냄
- $ x^{(0)} $ 에서의 함수 $f(x)에 대한 second-order Taylor series appoximation

$$
    f(x) \approx f(x^{(0)}) + (x - x^{(0)})^{\top}g + \frac{1}{2}(x - x^{(0)})^{\top}H(x - x^{(0)})
$$

- $ g $ : gradient at $ x^{(0)} $
- $ H $ : Hessian at $ x^{(0)} $
- learning rate $ \epsilon $을 통해 새로운 점 $ x^{(0)} - \epsilon g $를 나타내면

$$
    f(x^{(0)} - \epsilon g) \approx f(x^{(0)}) - \epsilon g^{\top}g + \frac{1}{2}\epsilon^2 g^{\top}Hg
$$

- 1번 항 : original value of the function
- 2번 항 : gradient descent에 의한 최소화
- 3번 항 : 함수의 curvature에 따른 보정


- 3번 항($ \frac{1}{2}\epsilon^2 g^{\top}Hg $)이 매우 큰 값을 갖으면? $\to$ gradient descent step은 위를 향함(잘못된 방향)
- $ g^{\top}Hg $이 0이거나 negative이면? $\to$ gradient descent step은 항상 아래를 향함(올바른 방향)


- $ g^{\top}Hg $에 positive일 경우, Taylor series approximation 된 함수를 줄일 수 있는 최적의 $ \epsilon $ :

$$
    \epsilon^* = \frac {g^{\top}g} {g^{\top}Hg}
$$

- 최악의 경우, $ g $와, maximal eigenvalue $ \lambda_{\max} $와 대응되는 $ H $의 eigenvector가 align될 경우 $ \to \epsilon^* = \frac {1}{\lambda_{\max}} $ 
- **결론** : 최소화하고자 하는 함수가 quadratic 함수로 근사될 수 있으면, Hessian의 eigenvalues는 learning rate의 크기를 결정함

<br />
p6:

- second derivative는 critical point가 local minimum인지, local maximum인지, 혹은 saddle point인지를 결정할 수 있음
    - *second derivative test*
    - $ f'(x) = 0 \text{ and } f''(x) > 0 \to $ : local minimum
    - $ f'(x) = 0 \text{ and } f''(x) < 0 \to $ : local minimum
    - $ f'(x) = 0 \text{ and } f''(x) = 0 \to $ : saddle point 혹은 part of flat region
    
<br />
p7:

- multiple dimension에서, 함수에 대한 입력의 모든 second derivatives를 구해야 함
- Hessian matrix의 eigendecomposition을 이용하면, multiple dimension에서의 second derivate test로 일반화할 수 있음
- critical point($\triangledown_xf(x)$)에서 Hessian matrix의 eigenvalues를 통해, 해당 critical point가 local minimum인지, local maximum인지, 혹은 saddle point인지 결정할 수 있음
- Hessian이 **positive definite**인 경우(모든 eigenvalue 들이 positive인 경우) $ \to $ local minimum
- Hessian이 **negative definite**인 경우(모든 eigenvalue 들이 negative인 경우) $ \to $ local maximum
- 하나 이상의 eigenvalue가 positive이고, 하나 이상의 eigenvalue가 negative인 경우 $ \to $ inconclusive
    - 함수 $ f(x) $의 한 cross section에서는 local maximum이고,
    - 함수 $ f(x) $의 다른 cross section에서는 local minimum이 됨

![Figure 4.5](https://github.com/SCPSAI/Deep-Learning-Book/blob/master/images/fig_4_5.PNG?raw=true)

- 하나 이상의 eigenvalue가 0이지만, 나머지가 모두 동일한 부호의 non-zero eigenvalue를 갖는 경우 $ \to $ inconclusive

<br />
p8:

- multiple dimensions에서, 한 점에서 매우 다양한 서로다른 second derivatives들이 있을 수 있음
    - 각 방향에 대한 서로 다른 second derivative가 존재하기 때문
- Hessian의 **condition number** : second derivative가 얼마나 많은지 측정
- Hessian이 poor condition number를 갖을 경우 $ \to $ gradient descent가 재대로 수행되지 않음
    - 한 방향에서는 derivative가 빠르게 증가하지만, 다른 방향에서는 느리게 증가하기 때문
    - gradient descent는 이러한 변화를 인식하지 못하므로, (느리게 증가한) 특정 방향을 우선적으로 움직여야 한다는 것을 알지 못함
    - 이러한 문제는 좋은 step size를 선택하는 것을 어렵게 만듬
    - step size를 너무 작게 할 경우, 특정 방향에서 너무 느리게 움직임에 따라 gradient descent에 문제가 발생함

![Figure 4.6](https://github.com/SCPSAI/Deep-Learning-Book/blob/master/images/fig_4_6.PNG?raw=true)

<br />
p9:

- 상기와 같은 문제(issue)들은 Hessian matrix 정보를 이용해서 search를 guide함으로써 해결할 수 있음 $ \to $ *Newton's method*
- Newton's method는 점 $ x^{(0)} $ 주변에서의 $ f(x) $를 근사하기 위한 second-order Taylor series expansion를 사용함

$$
    f(x) \approx f(x^{(0)}) + (x - x^{(0)})^{\top} \triangledown_xf(x^{(0)}) + \frac{1}{2}(x-x^{(0)})^{\top}H(f)(x^{(0)})(x-x^{(0)})
$$

- critical point

$$
    x^* = x^{(0)} - H(f)(x^{(0)})^{-1}\triangledown_xf(x^{(0)})
$$

- 함수 $ f $가 positive definite quadratic function이면
    - Newton's method는 한번에 이동(jump)해서 함수의 minimum을 구할 수 있음
- 함수 $ f $가 quadratic이 아니지만, 국소적으로(locally) positive definite quadratic으로 근사할 수 있으면
    - Newton's method를 국소적으로 여러번 적용해서 반복적으로(iteratively) 수정해서 함수의 minimum을 구함
    - gradient descent보다 빠르게 critical point를 찾을 수 있음
- Newton's method의 이러한 성질은 local minimum 근처에서는 유용함
- 반면, saddle point 근처에서는 유해한 성질임
- 즉, Newton's method는 critical point가 minimum인 경우에만 적절히(올바르게) 사용될 수 있음

<br />

p10(**page 92**):

- *first-ordier optimization algorithms* : gradient descent와 같이, 함수의 gradient만 사용해서 최적화하는 알고리즘
- *second-order optimization algorithms* : Newton's method와 같이, gradient와 Hessian matrix를 모두 사용해서 최적화하는 알고리즘

<br />
p11:

- 본 교재에서 다룬 optimization algorithm들은 매우 다양한 함수에 적용 가능함
- 그러나, deep learning에서 사용하는 함수들은 매우 복잡하기 때문에, 최적점을 보장하지는 못함
- 대부분의 다른 분야에서의 지배적인 optimzation 방법은 제한된 함수에서의 optimization 방법을 설계하는 것!

<br />
p12:

- deep learning 관점에서, 함수를 *Lipschitz continous* 혹은 *Lipschitz continous derivatives*로 제한해서 optimization을 보장함
- *Lipschitz continous* 함수란? : 함수 f의 변화율을 *Lipschitz constant* $ \mathcal{L} $로 제한한 함수

$$
    \forall_x, \forall_y, |f(x)-f(y)| \leq \mathcal{L}||x-y||_2
$$

<br />

p13(**93page**):

- *convex optimization* : convex function 이라는 strong restriction 하에서의 optimization
    - convex function에서의 Hessian은 모든 구간에서 positive semidefinite임
    - saddle point가 없고, local minimum이 global minimum과 같음 $ \to $ optimization이 쉽게 잘 동작함
    - 그러나, deep learning 에서의 대부분의 문제들은 convex optimization으로 표현하기 어려움
        - deep learning algorithms의 일부분(subroutine)에서만 적용 가능함
    - convex optimization 의 분석을 통한 아이디어들은 deep learning algorithm들의 convergence를 증명하는데 유용함

## 4.4 Constrained Optimization

p1:

- *constrained optimization* : 집합 $ \mathbb{S} $ 내의 x에 대한, 함수 $ f(x) $를 최소화, 최대화하는 최적화 기법
    - 목적함수 외에 파라미터가 만족해야 할 별도의 제약조건이 있는 경우의 최적화 기법
- *feasible* points : 집합 $ \mathbb{S} $ 내 점 x

<br />
p2: 

- norm constraint를 부과할 수 있음
- $ ||x|| \leq 1 $

<br />
p3:

- simple approach to contrained optimization : 
    - small constant step size $ \epsilon $을 사용할 경우, 먼저 gradient step을 구한 후, 집합 $ \mathbb{S} $ 내 점 x에 투영(project)
    - line search를 사용할 경우, step size $ \epsilon $과 집합 $ \mathbb{S} $ 내 점에 대해 최적의 함수를 갖는 파라미터 설정
        - step이나 line search를 적용하기 전, gradient를 feasible region의 tangent space로 투영(project)하면 효과적임
        
<br />
p4:

- sophisticated approach :
    - original, constrained optimization problem 으로 변환할 수 있는 서로다른 unconstrained optimization problem을 설계(design)
    - ex) $ x \in \mathbb{R}^2 $이고, x가 unit $ L^2 $ norm으로 constrained 될 때,
        - 함수 $ g(\theta) = f([\cos \theta, \sin \theta]^{\top}) $를 $ \theta $에 대해 최적화한 후, 파라미터 $ x $를 $ [\cos \theta, \sin \theta] $로 회귀시키는 방법
  
<br />

p5(**page 94**):

- *Karush-kuhn-Tucker*(KKT) approach :
    - 입력에 대한 등식(equality), 부등식(inequality) 제한조건이 있는 경우의 최적화 방법
    - constrained optimization의 가장 일반적인(general) 솔루션(solution)을 제공함
    - *generalized Lagrangian*, *generalized Lagrange function*

<br />
p6:

- 등식과 부등식으로 제한조건 $ \mathbb(S) $를 정의
    - $ \mathbb{S} = \{x | \forall i, g^{(i)}(x) = 0 \text{ and } \forall j, h^{(j)}(x) \leq 0 \} $
        - $m$개의 등식 제한조건(*eqaulity constraints*) $g^{(i)}$
        - $n$개의 부등식 제한조건(*ineqaulity constraints* $h^{(j)}$

- 라그랑주 승수법(*Lagrange multipliers*) : 등식(equality) 제한조건만 있는 경우의 최적화 방법

<br />
p7:

- KKT multipliers : $\lambda_i$, $\alpha_j$
- generalized Lagrangian 정의: 

$$
    L(x, \lambda, \alpha) = f(x) + \sum_i \lambda_i g^{(i)}(x) + \sum_j \alpha_j h^{(j)}(x)
$$


<br />
p8:

- constrained minimization problem $ \to $ generalized Lagrangian의 unconstrained optimization으로 풀 수 있음
- 하나 이상의 feasible point가 존재하고, $ f(x) $가 $ \infty $를 허용하지 않을 때,

$$
    \underset{x}{\operatorname{min}} \underset{\lambda}{\operatorname{max}} \underset{\alpha, \alpha \geq 0}{\operatorname{max}} L(x, \lambda, \alpha)
$$

- 상기의 식은 하기의 식과 동일한 optimal objective function과 optimal points x로 구성된 집합을 갖음

$$
    \underset{x \in \mathbb{S}}{\operatorname{min}} f(x)
$$

- 하기의 조건을 만족할 경우, 상기의 식은 유효

$$
    \underset{\lambda}{\operatorname{max}} \underset{\alpha, \alpha \geq 0}{\operatorname{max}} L(x, \lambda, \alpha) = f(x)
$$

- 하기의 조건을 위반할 경우, 상기의 식은 유효하지 않음

$$
    \underset{\lambda}{\operatorname{max}} \underset{\alpha, \alpha \geq 0}{\operatorname{max}} L(x, \lambda, \alpha) = \infty
$$

- 위 속성들은 infeasible point가 최적 해가 아니고, feasible points 내의 최적 해가 변경되지 않도록 보장함?
> these properties guarantee that no infeasible point will ever be optimal, and that the optimum within the feasible points is unchanged.

<br />

p9(**page 95**):

- 함수의 maximization을 하고자 할 경우, generalized Lagrange function을 $ -f(x) $로 구성

$$
    \underset{x}{\operatorname{min}} \underset{\lambda}{\operatorname{max}} \underset{\alpha, \alpha \geq 0}{\operatorname{max}} -f(x) + \sum_i \lambda_i g^{(i)}(x) + \sum_j \alpha_j h^{(j)}(x)
$$

- 다음 식으로 변경 가능

$$
    \underset{x}{\operatorname{max}} \underset{\lambda}{\operatorname{min}} \underset{\alpha, \alpha \geq 0}{\operatorname{min}} f(x) + \sum_i \lambda_i g^{(i)}(x) - \sum_j \alpha_j h^{(j)}(x)
$$

- 모든 $ \lambda_i $의 부호에 대해 최적화가 가능하므로, 등식 제한조건(equality constraints)에 해당하는 항의 부호는 - 로 바뀌지 않아도 문제되지 않음

<br />
p10:

- $h^{(i)}(x^*) = 0$인 경우, $ h^{(i)}(x) $ 가 *active* 임
- 만약 제한조건이 활성화(active)되지 않을 경우, 
    - contraints가 제거된 경우, local solution으로 유지됨????
> If a constraint is not active, then the solution to the problem found using that constraint would remain at least a local solution if that constraint were removed.
- inactive constraint가 다른 solution들을 배제할 수 있음
    - ex1) convex problem :
        - constraints를 통해 제거된 globally optimal points region
    - ex2) non-convex problem :
        - inactive된 constraints를 통해 배제된 더 좋은 local stationary points


- <span style="color:red">**추가 해석 및 학습 필요...**</span>

<br />
p11:

- *Karush-Kuhn-Tucker (KKT) conditions* :
    - generalized Lagrangian에 대한 gradient가 0
    - x와 KKT multipliers에 대한 모든 조건을 만족
    - $ \alpha \odot h(x) = 0 $

<br />
p12:

- KKT approach에 대한 자세한 정보는 Nocedal and Wright (2006) 참조

## 4.5 Example: Linear Least Squares

p1:

- **문제** : 다음 함수를 최소화하는 변수 $ x $를 찾고자 함

$$
    f(x) = \frac {1}{2} ||Ax-b||_2^2
$$

- 상기의 문제는 specialized linear algebra algorithms을 통해 효과적으로 풀 수 있음
- 그러나, 본 절에서는 gradient-based optimization을 통해 푸는 방법에 대해 소개함

<br />
p2:

- gradient 계산

$$
    triangledown_xf(x) = A^{\top}(Ax-b) = A^{\top}Ax - A^{\top}b
$$

<br />
p3:

- 계산된 gradient의 반대 방향으로 이동

![Alg 4.1](https://github.com/SCPSAI/Deep-Learning-Book/blob/master/images/alg_4_1.PNG?raw=true)


<br />
p4:

- **Newton's method를 이용한 optimization**
    - 함수 $ f(x) = \frac {1}{2} ||Ax-b||_2^2 $는 quadratic 이므로 적용 가능
    - Newton's method를 통한 optimization은 global minimum에 single step으로 수렴
    
<br />
p5:

- **constrainted optimization**
- constraint : $ x^{\top}x \leq 1 $
- Lagrangian : 

$$
    L(x, \lambda) = f(x) + \lambda \left ( x^{\top}x - 1 \right )
$$

- 문제의 풀이

$$
    \underset{x}{\operatorname{min}} \underset{\lambda, \lambda \geq 0}{\operatorname{max}} L(x, \lambda)
$$

<br />
p6 :

- unconstrained least squares problem에서 smallest-norm solution은 Moore-Penrose pseudoinverse ($ x = A^{+}b $)를 이용해서 풀 수 있음
    - 만약, 점 x가 feasible이면, constrained problem에 대한 solution
    - 만약, 점 x가 infeasible이면, constraint가 active되는 solution을 찾아야함

- x에 대한 Lagrangian을 미분 적용

$$
    A^{\top}Ax - A^{\top}b + 2 \lambda x = 0
$$

- 최적해 x를 구하면

$$
    x = (A^{\top}A + 2 \lambda I)^{-1} A^{\top}b
$$

- 결과가 constraint를 준수(obey)하도록, $ \lambda $의 크기를 결정할 필요가 있음
    - $ \lambda $에 대해 gradient ascent를 적용해서 구할 수 있음
    
$$
    \frac {\partial} {\partial \lambda}L(x, \lambda) = x^{\top}x - 1
$$

- $ x $가 올바른 norm을 갖고, $ \lambda $에 대한 derivative가 0이 될 때까지 gradient ascent 반복

<br />

p7(**page97**):

- 여기까지 machine learning 알고리즘을 개발하기 위한 수학적인 사전 지식에 대한 소개를 마무리함
- 본격적인 learning system을 구축하고 분석할 준비를 완료함
